In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType, TimestampType
from pyspark.sql import Row
from pyspark.sql.functions import regexp_replace, col, trim, sum, when, to_timestamp, substring, when, count, year, current_date, upper
import re

In [0]:
print(">>> Criando schema silver no catálogo de dados, caso ainda não exista...", end=" ")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS silver_layer");
print("OK!")

In [0]:
%run /Workspace/Users/douglasfmedeiros@outlook.com/desafio_neurotech/functions/silver_functions

In [0]:
# Lista todas as tabelas existentes na camada bronze
tabelas = spark.catalog.listTables("bronze_layer");
tabelas = [t for t in tabelas if t.name != "_sqldf"]

# A primeira análise a ser feita é se todas as tabelas importadas na camada bronze (brutas) possuem a mesma quantidade de colunas
print('================================== VERIFICAÇÃO - QTD COLUNAS DAS TABELAS ==================================')
print('>>> Verificando o número de colunas de cada tabela...\n')

for tabela in tabelas:

    table_name = tabela.name

    dados = spark.read.table(f"bronze_layer.{table_name}");

    # Verificar a quantidade de colunas de cada tabela
    print(f'Tabela: {table_name} / Qtd Colunas: {len(dados.columns)}')

print('')
print('>>> Na tabela IPTU_2024 há uma coluna adicional, vamos investigar se existem colunas diferentes entre as tabelas...')
print('===========================================================================================================')
print('')

In [0]:
print('>>> Importando todas as tabelas da camada bronze...', end=' ')
for tabela in tabelas:
    nome_variavel = tabela.name.replace('iptu', 'dados')
    globals()[nome_variavel] = spark.read.table(f'bronze_layer.{tabela.name}')

dfs = [eval(t.name.replace('iptu', 'dados')) for t in tabelas]
nomes = [tabela.name for tabela in tabelas]
print('OK!')

In [0]:
print('================================== VERIFICAÇÃO - COLUNAS EXCLUSIVAS ==================================')
print('>>> Verificando se há colunas diferentes entre os dataframes...\n')

# Função para encontrar colunas exclusivas de cada DataFrame
for i, df_base in enumerate(dfs):
    colunas_base = set(df_base.columns)
    
    # Junta todas as colunas dos outros DataFrames
    colunas_outras = set().union(*[set(dfs[j].columns) for j in range(len(dfs)) if j != i])
    
    # Calcula colunas exclusivas
    exclusivas = colunas_base - colunas_outras
    
    print(f"Colunas exclusivas de {nomes[i]}: {sorted(exclusivas)}")

print('')
print('>>> Como esperado, há colunas adicionais na tabela IPTU_2024, vamos ajustar isso...', end=" ")

# Ajustando colunas da tabela de IPTU de 2024
dados_2024 = dados_2024.withColumnRenamed('quant pavimentos', 'quantidade de pavimentos').withColumnRenamed('valor IPTU', 'valor cobrado de IPTU').drop('_id')

print('OK!')

print('======================================================================================================')

In [0]:
dfs = [eval(t.name.replace('iptu', 'dados')) for t in tabelas]

In [0]:
print('>>> Analisando os tipos de dados das tabelas da camada bronze (inferidas pelo spark), podemos perceber algumas inconsistências, como, por exemplo, a coluna ÁREA CONSTRUÍDA estar como string, quando deveria ser uma informação numérica. Durante o processamento de dados adiante, isso será ajustado para todas as colunas. Os tipos de todas as colunas podem ser observados abaixo: \n')

In [0]:
dados_2024.dtypes

In [0]:
for df, nome in zip(dfs, nomes):

    table_name = nome

    print(f'========================================== PROCESSAMENTO DE DADOS [Tabela {table_name}] ==========================================\n')

    # Ajuste na nomenclatura das colunas
    print('Ajustando os nomes das colunas...', end=" ")

    df = df.toDF(*[limpar_nome_coluna(c) for c in df.columns])

    print('OK!')    

    # Removendo espaços (lixo)
    print('Removendo espaços desnecessários do conteúdo da tabela...', end=" ")

    df = df.select([
        trim(col(c)).alias(c) if isinstance(df.schema[c].dataType, StringType) else col(c)
        for c in df.columns
    ])

    print('OK!')

    # Ajustando coluna TIPO_DE_CONTRIBUINTE
    df = df.replace(["", " "], "Não informado", subset=["TIPO_DE_CONTRIBUINTE", "COMPLEMENTO"])

    # Ajustando o nome do bairro
    df = df.withColumn("BAIRRO", upper(df["BAIRRO"]))
    
    print('Ajustando os tipos de dados...', end=" ")
    # Ajustando os tipos de dados
    df = (
        df.withColumn('NUMERO_DO_CONTRIBUINTE', col('NUMERO_DO_CONTRIBUINTE').cast('string'))
        .withColumn('ANO_DO_EXERCICIO', col('ANO_DO_EXERCICIO').try_cast('double').try_cast('integer'))
        #.withColumn('DATA_DO_CADASTRAMENTO', col('DATA_DO_CADASTRAMENTO').cast('string'))
        .withColumn('TIPO_DE_CONTRIBUINTE', col('TIPO_DE_CONTRIBUINTE').cast('string'))
        .withColumn('CPFCNPJ_MASCARADO_DO_CONTRIBUINTE', col('CPFCNPJ_MASCARADO_DO_CONTRIBUINTE').cast('string'))   
        .withColumn('LOGRADOURO', col('LOGRADOURO').cast('string'))   
        .withColumn('NUMERO', col('NUMERO').try_cast('double').try_cast('integer'))  
        .withColumn('COMPLEMENTO', col('COMPLEMENTO').cast('string')) 
        .withColumn('BAIRRO', col('BAIRRO').cast('string')) 
        .withColumn('CIDADE', col('CIDADE').cast('string')) 
        .withColumn('ESTADO', col('ESTADO').cast('string')) 
        .withColumn('FRACAO_IDEAL', regexp_replace(col('FRACAO_IDEAL'), ",", ".").try_cast('double')) 
        .withColumn('AREA_TERRENO', regexp_replace(col('AREA_TERRENO'), ",", ".").try_cast('double')) 
        .withColumn('AREA_CONSTRUIDA', regexp_replace(col('AREA_CONSTRUIDA'), ",", ".").try_cast('double')) 
        .withColumn('AREA_OCUPADA', regexp_replace(col('AREA_OCUPADA'), ",", ".").try_cast('double')) 
        .withColumn('VALOR_DO_M2_DO_TERRENO', regexp_replace(col('VALOR_DO_M2_DO_TERRENO'), ",", ".").try_cast('double')) 
        .withColumn('VALOR_DO_M2_DE_CONSTRUCAO', regexp_replace(col('VALOR_DO_M2_DE_CONSTRUCAO'), ",", ".").try_cast('double')) 
        .withColumn('ANO_DA_CONSTRUCAO_CORRIGIDO', col('ANO_DA_CONSTRUCAO_CORRIGIDO').try_cast('double').try_cast('integer')) 
        .withColumn('QUANTIDADE_DE_PAVIMENTOS', col('QUANTIDADE_DE_PAVIMENTOS').try_cast('double').try_cast('integer')) 
        .withColumn('TIPO_DE_USO_DO_IMOVEL', col('TIPO_DE_USO_DO_IMOVEL').cast('string')) 
        .withColumn('TIPO_DE_PADRAO_DA_CONSTRUCAO', col('TIPO_DE_PADRAO_DA_CONSTRUCAO').cast('string')) 
        .withColumn('FATOR_DE_OBSOLESCENCIA', regexp_replace(col('FATOR_DE_OBSOLESCENCIA'), ",", ".").try_cast('double')) 
        .withColumn('ANO_E_MES_DE_INICIO_DA_CONTRIBUICAO', col('ANO_E_MES_DE_INICIO_DA_CONTRIBUICAO').cast('string')) 
        .withColumn('VALOR_TOTAL_DO_IMOVEL_ESTIMADO', regexp_replace(col('VALOR_TOTAL_DO_IMOVEL_ESTIMADO'), ",", ".").try_cast('double')) 
        .withColumn('VALOR_COBRADO_DE_IPTU', regexp_replace(col('VALOR_COBRADO_DE_IPTU'), ",", ".").try_cast('double')) 
        .withColumn('CEP', col('CEP').try_cast('double').try_cast('integer')) 
        .withColumn('REGIME_DE_TRIBUTACAO_DO_IPTU', col('REGIME_DE_TRIBUTACAO_DO_IPTU').cast('string')) 
        .withColumn('REGIME_DE_TRIBUTACAO_DA_TRSD', col('REGIME_DE_TRIBUTACAO_DA_TRSD').cast('string')) 
        .withColumn('TIPO_DE_CONSTRUCAO', col('TIPO_DE_CONSTRUCAO').cast('string')) 
        .withColumn('TIPO_DE_EMPREENDIMENTO', col('TIPO_DE_EMPREENDIMENTO').cast('string')) 
        .withColumn('TIPO_DE_ESTRUTURA', col('TIPO_DE_ESTRUTURA').cast('string')) 
        .withColumn('CODIGO_LOGRADOURO', col('CODIGO_LOGRADOURO').try_cast('double').try_cast('integer')) 
    )
    print('OK!')  

    # Ajustar tipo de dados da data (apenas para os casos em que é string)  
    if(isinstance(df.schema["DATA_DO_CADASTRAMENTO"].dataType, StringType)):
        print('Ajustando o formato de dados da coluna DATA_DO_CADASTRAMENTO...', end=' ')
        
        df = df.withColumn('DATA_DO_CADASTRAMENTO', to_timestamp(substring(col("DATA_DO_CADASTRAMENTO"), 1, 23), "yyyy/MM/dd HH:mm:ss.SSS"))
    
        print('OK!')

    # Verificando dados duplicados
    print('Removendo dados duplicados...', end=' ')
    df = df.dropDuplicates()
    print('OK!')

    print('\n------------------------------------ ANÁLISE DE REGISTROS NULOS ------------------------------------\n')
    
    print('Analisando a quantidade de registros nulos por coluna...\n')

    analise_nulos(df)

    print('\n>>> Como a quantidade de registros nulosé inferior a 1%, vamos remover os registros nulos')

    # Removendo linhas que possuem valores nulos
    print('Removendo registros com valores nulos...', end=" ")
    df = df.na.drop()  
    print('OK!')

    print('\n---------------------------------- ANÁLISE DE QUALIDADE DOS DADOS ----------------------------------\n')
    
    # Análise da qualidade dos dados
    df = analise_qualidade_dados(df)    

    print('\n---------------------------------------- INGESTÃO DOS DADOS ----------------------------------------\n')

    ingestao_dados(df, table_name, 'silver')

    print('==================================================== PROCESSAMENTO CONCLUÍDO ==================================================\n')

In [0]:
print(f'////////////////////////////////////////// INGESTÃO DA CAMADA SILVER FINALIZADA //////////////////////////////////////////')